# Simulazione modello AIM

In [4]:
import numpy as np
import numba
from numba import njit, prange
from scipy.io import savemat
import matplotlib.pyplot as plt
from simula1d import *

Ho un po' di funzioni che fanno evoluzione temporale, scrivo qua sotto per non dimenticare:
- evoluzione_profili = profili e basta solo alla fine (non ha senso usarla)
- evoluzione_filmino = salva densità e magnetizzazione che sono effettivamente da controllare, secondo quello che dice array
- evoluzione_track = tiene traccia casualmente di alcune particelle per osservare meglio la dinamica
- evoluzione_scorporata = scorpora il meccanismo di flip da quello della diffusione

Compilazioni preliminari per velocizzare

In [5]:
L = 5
n = 200
particles = np.empty((n,2), dtype=np.int64)
particles[:,0] = np.random.randint(0, L, size=n)
particles[:,1] = 2*np.random.randint(0,2,size=n) - 1

density = np.zeros(L, np.int32)
magnet  = np.zeros(L, np.int32)

# popola matrici iniziali
for k in range(n):
    x, s = particles[k]
    density[x] += 1
    magnet[x]  += s

array = np.arange(0,0)
rateflip_numba(L,1,1,1,1,1,magnet,density)
evoluzione_filmino(L,1,1,1,particles,10,1,1,array) #compila

(array([], shape=(0, 5), dtype=float64),
 array([], shape=(0, 5), dtype=float64),
 array([0.2 , 0.12, 0.24, 0.23, 0.17, 0.14, 0.1 , 0.13, 0.13, 0.19]))

In [6]:
# Parametri delle simulazioni
size = [50, 75, 100, 200, 300, 400, 500]
D, epsilon  = 1.0, 0.9
t_max       = 15_000_000
freq_frames = 10_000
log_array = build_log_indices(t_max, freq_frames)
num = 21 #numero di simulazioni da lanciare per ogni taglia

dens_init = 5 # ci sono cluster a basse T?
T         = 0.2 #temperature
betas     = 1/T
gamma    = 0 # grado di località
metodi    = 1 # smetodo di "località"


for j in range(1, num):
    for i, L in enumerate(size):
        n = int(L * dens_init) #n_particelle data densità
    
        # genera particelle dato n
        particles = np.empty((n,2), dtype=np.int64)
        particles[:,0] = np.random.randint(0, L, size=n)
        particles[:,1] = 2*np.random.randint(0,2,size=n) - 1

        # chiama la funzione numba (già compilata)
        frames_d, frames_m, order_param = evoluzione_filmino(
            L, D, epsilon,
            betas, particles, t_max,
            gamma, metodi, log_array
        )

        # salvataggio per successiva visualizzazione (anche in Matlab)

        savemat(f'1Dcoarse{j}T{1/betas:.3f}L{L}' + f'frames_d'+'.mat', {'frames_d': frames_d})
        savemat(f'1Dcoarse{j}T{1/betas:.3f}L{L}' + f'frames_m'+'.mat', {'frames_m': frames_m})
        savemat(f'1Dcoarse{j}T{1/betas:.3f}L{L}' + f'_order_param'+'.mat', {'order_param': order_param})